In [1]:
!pip install requests pandas sqlalchemy pyodbc

  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/d0/ff/5746886a796799d45285573c8a6564b4b84c730142ab974d7a3f7bacee6c/SQLAlchemy-2.0.31-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pyodbc from https://files.pythonhosted.org/packages/71/6e/6b8ec142713bbb8e34da6b73cf281699904823e1a61f9a78b6cbda92301a/pyodbc-5.1.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.6.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/53/80/3d94d5999b4179d91bcc93745d1b0815b073d61be79dd546b840d17adb18/greenlet-3.0.3-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.1 MB 3.1 MB/s e


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# Extract Data from API
import requests
#library to handle HTTP requests
url = "https://us-hospitals.p.rapidapi.com/?ccn=670068"  # API endpoint obtained from RapidAPI
headers = {
    "X-RapidAPI-Host": "us-hospitals.p.rapidapi.com", #specifies the host API
    "X-RapidAPI-Key": "01a8db32e5msh88269d4cb8f2b96p10e163jsnfe7ac76437ab"  # RapidAPI key
}

response = requests.get(url, headers=headers)
data = response.json()

# Displying the obtained Data
print(data)

[{'Provider CCN': 670068, 'Hospital Name': 'Texas Health Presbyterian Hospital Flower Mound', 'Alias': 'Presbyterian Hosp Flower Mound', 'Url': 'http://www.texashealthflowermound.com/', 'Phone': '(469)322-7000', 'Service': 'General medical and surgical', 'Type': 'General Short Term', 'Approval Codes': 'Medicare certification by the Centers for Medicare and Medicaid Services,,The Joint Commission accreditation,,The Joint Commission accreditation,', 'Street Address': '4400 Long Prairie Road', 'City': 'Flower Mound', 'State Code': 'TX', 'Zip Code': '75028-1892', 'County': 'Denton', 'Medicare CBSA Number': 19124, 'Rural Versus Urban': 'U', 'Control': 'Partnership,Investor-owned,for-profit', 'Number of Beds': 99, 'Parent Unit': None, 'Affiliated Units': None, 'System': None}]


In [4]:
# Load data into Dataframe 
import pandas as pd

In [5]:
# Data is a list of Dictionaries
df = pd.DataFrame(data)
print(df.head())

   Provider CCN                                    Hospital Name  \
0        670068  Texas Health Presbyterian Hospital Flower Mound   

                            Alias                                     Url  \
0  Presbyterian Hosp Flower Mound  http://www.texashealthflowermound.com/   

           Phone                       Service                Type  \
0  (469)322-7000  General medical and surgical  General Short Term   

                                      Approval Codes          Street Address  \
0  Medicare certification by the Centers for Medi...  4400 Long Prairie Road   

           City State Code    Zip Code  County  Medicare CBSA Number  \
0  Flower Mound         TX  75028-1892  Denton                 19124   

  Rural Versus Urban                                Control  Number of Beds  \
0                  U  Partnership,Investor-owned,for-profit              99   

  Parent Unit Affiliated Units System  
0        None             None   None  


In [8]:
# Transform Data:
df['Number of Beds'] = df['Number of Beds'].astype(int)
print(df.head())

   Provider CCN                                    Hospital Name  \
0        670068  Texas Health Presbyterian Hospital Flower Mound   

                            Alias                                     Url  \
0  Presbyterian Hosp Flower Mound  http://www.texashealthflowermound.com/   

           Phone                       Service                Type  \
0  (469)322-7000  General medical and surgical  General Short Term   

                                      Approval Codes          Street Address  \
0  Medicare certification by the Centers for Medi...  4400 Long Prairie Road   

           City State Code    Zip Code  County  Medicare CBSA Number  \
0  Flower Mound         TX  75028-1892  Denton                 19124   

  Rural Versus Urban                                Control  Number of Beds  \
0                  U  Partnership,Investor-owned,for-profit              99   

  Parent Unit Affiliated Units System  
0        None             None   None  


In [9]:
# Standardize text fields
df['Hospital Name'] = df['Hospital Name'].str.strip()
df['Alias'] = df['Alias'].str.strip()
df['City'] = df['City'].str.strip()
df['Control'] = df['Control'].str.strip()

# Handle missing values
df.fillna({'Parent Unit': 'Not Available', 'Affiliated Units': 'Not Available', 'System': 'Not Available'}, inplace=True)

# Normalize phone numbers
df['Phone'] = df['Phone'].apply(lambda x: f"+1-{x[:3]}-{x[3:6]}-{x[6:]}")

# Display the transformed DataFrame
print(df.head())

   Provider CCN                                    Hospital Name  \
0        670068  Texas Health Presbyterian Hospital Flower Mound   

                            Alias                                     Url  \
0  Presbyterian Hosp Flower Mound  http://www.texashealthflowermound.com/   

                Phone                       Service                Type  \
0  +1-(46-9)3-22-7000  General medical and surgical  General Short Term   

                                      Approval Codes          Street Address  \
0  Medicare certification by the Centers for Medi...  4400 Long Prairie Road   

           City State Code    Zip Code  County  Medicare CBSA Number  \
0  Flower Mound         TX  75028-1892  Denton                 19124   

  Rural Versus Urban                                Control  Number of Beds  \
0                  U  Partnership,Investor-owned,for-profit              99   

     Parent Unit Affiliated Units         System  
0  Not Available    Not Available  Not Av

In [11]:
!pip install sqlalchemy pyodbc


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from sqlalchemy import create_engine

# Load Data into Microsoft SQL Server
server = 'VINOTH\\SQLEXPRESS'  # server name
database = 'SQL Project'  # database name
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f'mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes'
engine = create_engine(connection_string)


In [22]:
# Loading DataFrame into SQL Table
df.to_sql('hospitals', con=engine, if_exists='replace', index=False)

1